In [23]:
! curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
! curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
! curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 57935  100 57935    0     0   401k      0 --:--:-- --:--:-- --:--:--  401k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   835  100   835    0     0   4465      0 --:--:-- --:--:-- --:--:--  4465
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   872  100   872    0     0   4295      0 --:--:-- --:--:-- --:--:--  4295


In [25]:
import pandas as pd
import numpy as np

In [26]:
pop = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')

In [27]:
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [28]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [29]:
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [38]:
merged = pd.merge(abbrevs,pop,left_on='abbreviation',right_on='state/region',how='outer')
merged = merged.drop('abbreviation',axis=1)
merged.head()

,state,state/region,ages,year,population
0,Alabama,AL,under18,2012,1117489.0
1,Alabama,AL,total,2012,4817528.0
2,Alabama,AL,under18,2010,1130966.0
3,Alabama,AL,total,2010,4785570.0
4,Alabama,AL,under18,2011,1125763.0


In [42]:
merged.isnull().any()

state            True
state/region    False
ages            False
year            False
population       True
dtype: bool

In [45]:
merged.isnull().sum()

state           96
state/region     0
ages             0
year             0
population      20
dtype: int64

In [46]:
merged[merged['population'].isnull()].head()

,state,state/region,ages,year,population
2448,NaN,PR,under18,1990,NaN
2449,NaN,PR,total,1990,NaN
2450,NaN,PR,total,1991,NaN
2451,NaN,PR,under18,1991,NaN
2452,NaN,PR,total,1993,NaN


In [49]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [73]:
merged.loc[merged['state/region']=='PR','state']='Puerto Rico'
merged.loc[merged['state/region']=='USA','state']='United States'
merged.isnull().any()

state           False
state/region    False
ages            False
year            False
population       True
dtype: bool

In [82]:
final = pd.merge(merged,areas,how='outer')
final.head()

,state,state/region,ages,year,population,area (sq. mi)
0,Alabama,AL,under18,2012,1117489.0,52423.0
1,Alabama,AL,total,2012,4817528.0,52423.0
2,Alabama,AL,under18,2010,1130966.0,52423.0
3,Alabama,AL,total,2010,4785570.0,52423.0
4,Alabama,AL,under18,2011,1125763.0,52423.0


In [83]:
final.isnull().any()

state            False
state/region     False
ages             False
year             False
population        True
area (sq. mi)     True
dtype: bool

In [87]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [89]:
final.dropna(inplace=True)
final.head()

,state,state/region,ages,year,population,area (sq. mi)
0,Alabama,AL,under18,2012,1117489.0,52423.0
1,Alabama,AL,total,2012,4817528.0,52423.0
2,Alabama,AL,under18,2010,1130966.0,52423.0
3,Alabama,AL,total,2010,4785570.0,52423.0
4,Alabama,AL,under18,2011,1125763.0,52423.0


In [91]:
data2010 = final.query("year==2010 & ages=='total'")
data2010.head()

,state,state/region,ages,year,population,area (sq. mi)
3,Alabama,AL,total,2010,4785570.0,52423.0
91,Alaska,AK,total,2010,713868.0,656425.0
101,Arizona,AZ,total,2010,6408790.0,114006.0
189,Arkansas,AR,total,2010,2922280.0,53182.0
197,California,CA,total,2010,37333601.0,163707.0


In [93]:
data2010.set_index('state',inplace=True)
density = data2010['population']/data2010['area (sq. mi)']

In [97]:
density.sort_values(ascending=False,inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [102]:
d = density.reset_index()
d.loc[d['state']=='Washington']

,state,0
24,Washington,94.557817
